In [145]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import date
from dateutil.relativedelta import *
import json
import os

import numpy as np

import re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait

from selenium.common.exceptions import TimeoutException

In [147]:
# Get MyFantasyLeague players' name, team name, position, and ages
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=players"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')
data = []
players = soup.find_all('player')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("name"), players[i].get("position"), players[i].get("team")]
    data.append(rows)
player_df = pd.DataFrame(data)
player_df.columns=['PlayerID','Name', 'Position', 'Team']

# Get ages
to_query_age = player_df
player_dobs  = pd.DataFrame()
if len(to_query_age)>0:
    # Break player list into chunks small enough for the API server
    n = 50  #chunk row size
    list_df = [to_query_age.PlayerID[i:i+n] for i in range(0,to_query_age.PlayerID.shape[0],n)]

    for i in range(len(list_df)):
        idList = ",".join(list_df[i])

        # Get playerProfiles
        urlString = f"https://api.myfantasyleague.com/2022/export?TYPE=playerProfile&P={idList}"
        response = requests.get(urlString)
        soup = BeautifulSoup(response.content,'xml')
        data = []
        profiles = soup.find_all('playerProfile')
        players = soup.find_all('player')
        for i in range(len(profiles)):
            rows = [profiles[i].get("id"), players[i].get("dob")]
            data.append(rows)
        data_df = pd.DataFrame(data)
        age = pd.DataFrame(columns=['PlayerID', 'DOB'])
        age['PlayerID'] = data_df[0]
        age['DOB'] = data_df[1]
        player_dobs = player_dobs.append(age)

# Convert string to datetime
player_dobs['DOB'] = pd.to_datetime(player_dobs['DOB'])
# Convert DOB to Age
today = date.today()
def age(born):
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
player_dobs['Age'] = player_dobs['DOB'].apply(age)

# Merge all dfs
player_df = player_df.merge(player_dobs, on='PlayerID', how='left')
player_df = player_df.drop(columns='DOB')

In [151]:
# Clean MFL data
mfl_df = player_df.copy()
# Select only relevant positions
mfl_df = mfl_df.loc[mfl_df['Position'].isin(['QB', 'WR', 'RB', 'TE', 'PK', 'Def'])]
mfl_df = mfl_df.reset_index(drop=True)

# Clean Name column
to_join = mfl_df['Name'].str.split(", ", n=1, expand=True)
to_join.columns = ['lname', 'fname']
to_join['Name'] = to_join['fname'] + " " + to_join['lname']
mfl_df['Name'] = to_join['Name']
# Change to Title Case
mfl_df.Name = mfl_df.Name.str.upper()
# Drop punctuation
mfl_df.Name = mfl_df.Name.str.replace(".", "")
mfl_df.Name = mfl_df.Name.str.replace(",", "")
mfl_df.Name = mfl_df.Name.str.replace("'", "")

# Clean position column
mfl_df['Position'] = mfl_df['Position'].replace('Def', 'DF')

# Clean Team column
mfl_df['Team'] = mfl_df['Team'].replace('FA*', 'FA')

# Change column names
mfl_df.columns = ['id_mfl', 'player', 'pos_mfl', 'team', 'age']
mfl_df


/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  from ipykernel import kernelapp as app


,id_mfl,player,pos_mfl,team,age
0,0501,BUFFALO BILLS,DF,BUF,NaN
1,0502,INDIANAPOLIS COLTS,DF,IND,NaN
2,0503,MIAMI DOLPHINS,DF,MIA,NaN
3,0504,NEW ENGLAND PATRIOTS,DF,NEP,NaN
4,0505,NEW YORK JETS,DF,NYJ,NaN
...,...,...,...,...,...
1093,15996,JALEN VIRGIL,WR,DEN,24.0
1094,15997,DANNY DAVIS,WR,GBP,24.0
1095,15999,RAMIZ AHMED,PK,GBP,27.0
1096,16000,PEYTON HENDERSHOT,TE,DAL,23.0


In [43]:
# Get OurLads data
# Set Selenium settings
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"
# Scrape web for stats
url = f"https://www.ourlads.com/nfldepthcharts/depthcharts.aspx"

PATH = "/Applications/chromedriver"
driver = webdriver.Chrome(service=Service(PATH), desired_capabilities=capa)
wait = WebDriverWait(driver, 20)
driver.get(url)

wait.until(EC.presence_of_element_located((By.XPATH, "//table[@id='ctl00_phContent_gvChart']")))
driver.execute_script("window.stop();")

ourlads_scrape = pd.read_html(driver.find_element(By.XPATH, value="//table[@id='ctl00_phContent_gvChart']").get_attribute("outerHTML"))
ourlads_scrape = ourlads_scrape[0]

In [90]:
# Clean ourlads_df data
df = ourlads_scrape.copy()
df = df[['Team', 'Pos', 'Player 1', 'Player 2','Player 3', 'Player 4', 'Player 5']]

# Transform columns into rows
df1 = df[['Team', 'Pos', 'Player 1']]
df1 = df1.rename(columns={'Player 1':'Player'})
df1['posRank'] = "1"

df2 = df[['Team', 'Pos', 'Player 2']]
df2 = df2.rename(columns={'Player 2':'Player'})
df2['posRank'] = "2"

df3 = df[['Team', 'Pos', 'Player 3']]
df3 = df3.rename(columns={'Player 3':'Player'})
df3['posRank'] = "3"

df4 = df[['Team', 'Pos', 'Player 4']]
df4 = df4.rename(columns={'Player 4':'Player'})
df4['posRank'] = "4"

df5 = df[['Team', 'Pos', 'Player 5']]
df5 = df5.rename(columns={'Player 5':'Player'})
df5['posRank'] = "5"

complete = pd.concat([df1, df2, df3, df4, df5], axis=0, ignore_index=True)

# Clean Position column
# Select only relevant positions
posList = ['LWR', 'RWR', 'SWR', 'TE', 'QB', 'RB', 'PK', 'PR', 'KR', 'RES']
ourlads_df = complete.loc[complete['Pos'].isin(posList)]
# Convert WR roles to "WR"
ourlads_df['Pos'].replace(["LWR", "RWR", "SWR"], "WR", inplace=True)
ourlads_df['posRank'] = ourlads_df['Pos'] + ourlads_df['posRank']
ourlads_df = ourlads_df.reset_index(drop=True)
ourlads_df.dropna(inplace=True)
ourlads_df.drop_duplicates(subset=['Player', 'Team', 'Pos'], inplace=True)

# Create columns for KRs and PRs
krs = ourlads_df.loc[ourlads_df.Pos=='KR']
krs = krs.drop(columns=['Pos'])
krs.columns = ['Team', 'Player', 'KR']
prs = ourlads_df.loc[ourlads_df.Pos=='PR']
prs = prs.drop(columns=['Pos'])
prs.columns = ['Team', 'Player', 'PR']
# Join pr and pk dfs back onto main ourlads df
ourlads_df = ourlads_df.merge(krs, how='left', on=['Player', 'Team']).merge(prs, how='left', on=['Player', 'Team'])
ourlads_df['KR'].fillna("NO", inplace=True)
ourlads_df['PR'].fillna("NO", inplace=True)

# Clean name column
names = ourlads_df['Player'].str.split(" ", n=2, expand=True)
names.columns = ['a', 'b', 'c']
names['a'] = names['a'].str.replace(",", "")
ourlads_df['Player'] = names['b'] + " " + names['a']
# Change to Upper Case
ourlads_df.Player = ourlads_df.Player.str.upper()
# Drop punctuation
ourlads_df.Player = ourlads_df.Player.str.replace(".", "")
ourlads_df.Player = ourlads_df.Player.str.replace(",", "")
ourlads_df.Player = ourlads_df.Player.str.replace("'", "")

# Change column names and order
ourlads_df = ourlads_df[['Player', 'Pos', 'Team', 'posRank', 'KR', 'PR']]
ourlads_df.columns = ['player', 'pos_ol', 'team', 'posRank', 'KR', 'PR']

# Remove separate rows for PRs and KRs
ourlads_df = ourlads_df.loc[(ourlads_df.pos_ol!="KR")]
ourlads_df = ourlads_df.loc[(ourlads_df.pos_ol!="PR")]

# Drop position column
ourlads_df.drop(columns=['pos_ol'], inplace=True)
ourlads_df

/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/travisharry/opt/anaconda3/envs/ffpred/lib/python3.7/site-packages/ipykernel_launcher.py:59: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions

,player,team,posRank,KR,PR
0,AJ GREEN,ARZ,WR1,NO,NO
1,MARQUISE BROWN,ARZ,WR1,NO,NO
2,RONDALE MOORE,ARZ,WR1,KR1,PR1
3,ZACH ERTZ,ARZ,TE1,NO,NO
4,KYLER MURRAY,ARZ,QB1,NO,NO
...,...,...,...,...,...
1147,JULIUS CHESTNUT,TEN,RB5,NO,NO
1148,CURTIS HODGES,WAS,TE5,NO,NO
1149,ELI WOLF,WAS,TE5,NO,NO
1150,JARET PATTERSON,WAS,RB5,KR3,NO


In [152]:
# Merge MyFantasyLeague and OurLads data
mfl_ol_df = mfl_df.merge(ourlads_df, how='left', on=['player', 'team'])
# Clean merged df
mfl_ol_df.loc[mfl_ol_df['pos_mfl']=='DF', 'posRank'] = "DF1"
mfl_ol_df['KR'].fillna("NO", inplace=True)
mfl_ol_df['PR'].fillna("NO", inplace=True)

# Clean posRanks
mfl_ol_df['posRank'] = mfl_ol_df['posRank'].map({
    'RES1':'RES',
    'RES2':'RES',
    'RES3':'RES',
    'RES4':'RES',
    'RES5':'RES',
    'QB1':'QB1', 
    'QB2':'QB2', 
    'QB3':'QB3', 
    'QB4':'QB3',
    'QB5':'QB3', 
    'RB1':'RB1', 
    'RB2':'RB2', 
    'RB3':'RB3', 
    'RB4':'RB3', 
    'RB5':'RB3',
    'RB6':'RB3', 
    'RB7':'RB3', 
    'WR1': 'WR1', 
    'WR2': 'WR1', 
    'WR3': 'WR1', 
    'WR4': 'WR2', 
    'WR5': 'WR2', 
    'WR6': 'WR2', 
    'WR7': 'WR3', 
    'WR8': 'WR3',
    'TE1':'TE1', 
    'TE2':'TE2', 
    'TE3':'TE3', 
    'TE4':'TE3', 
    'TE5':'TE3', 
    'PK1':'PK1', 
    'PK2':'PK2', 
    'PK3':'PK3',
    'DF1':'DF1'
    })

mfl_ol_df['RES'] = "NO"
mfl_ol_df.loc[mfl_ol_df['posRank']=="RES", 'RES'] = "RES"
mfl_ol_df.loc[mfl_ol_df.posRank.isna(), 'posRank'] = mfl_ol_df.loc[mfl_ol_df.posRank.isna(), 'pos_mfl'] + "3"
mfl_ol_df.loc[mfl_ol_df.posRank=="RES", 'posRank'] = mfl_ol_df.loc[mfl_ol_df.posRank=="RES", 'pos_mfl'] + "3"
# Specify current season
mfl_ol_df['season'] = 2022
mfl_ol_df

,id_mfl,player,pos_mfl,team,age,posRank,KR,PR,RES,season
0,0501,BUFFALO BILLS,DF,BUF,NaN,DF1,NO,NO,NO,2022
1,0502,INDIANAPOLIS COLTS,DF,IND,NaN,DF1,NO,NO,NO,2022
2,0503,MIAMI DOLPHINS,DF,MIA,NaN,DF1,NO,NO,NO,2022
3,0504,NEW ENGLAND PATRIOTS,DF,NEP,NaN,DF1,NO,NO,NO,2022
4,0505,NEW YORK JETS,DF,NYJ,NaN,DF1,NO,NO,NO,2022
...,...,...,...,...,...,...,...,...,...,...
1093,15996,JALEN VIRGIL,WR,DEN,24.0,WR2,NO,NO,NO,2022
1094,15997,DANNY DAVIS,WR,GBP,24.0,WR3,NO,NO,NO,2022
1095,15999,RAMIZ AHMED,PK,GBP,27.0,PK3,NO,NO,NO,2022
1096,16000,PEYTON HENDERSHOT,TE,DAL,23.0,TE3,NO,NO,NO,2022


In [210]:
# Get historical data
histdf = pd.read_csv('data_cleaned/modelSource.csv', index_col=0)

# Select only previous two years
histdf = histdf.loc[histdf['season']>=2020]

# Clean defense names in historical data
histdf.player = histdf.player.str.upper()
histdf


,season,week,team,player,age,pos,posRank,opponent,passA,passC,...,defSack_prior1_opp,defI_prior1_opp,defSaf_prior1_opp,defFum_prior1_opp,defBlk_prior1_opp,defT_prior1_opp,defPtsAgainst_prior1_opp,defPassYAgainst_prior1_opp,defRushYAgainst_prior1_opp,defYdsAgainst_prior1_opp
4749,2020.0,1.0,GBP,AARON RODGERS,37.0,QB,QB1,MIN,44.0,32.0,...,3.0000,1.0625,0.0625,0.8750,0.0625,0.1250,18.9375,233.5625,108.0000,341.5625
4750,2020.0,1.0,SEA,RUSSELL WILSON,32.0,QB,QB1,ATL,35.0,31.0,...,1.7500,0.7500,0.0000,0.5000,0.0625,0.1875,24.9375,244.8750,110.9375,355.8125
4751,2020.0,1.0,BUF,JOSH ALLEN,24.0,QB,QB1,NYJ,46.0,33.0,...,2.1875,0.7500,0.1250,0.5625,0.0625,0.3750,22.4375,236.1875,86.9375,323.1250
4752,2020.0,1.0,BAL,LAMAR JACKSON,23.0,QB,QB1,CLE,25.0,20.0,...,2.3750,0.8750,0.0000,0.3750,0.1250,0.0625,24.5625,216.8750,144.6875,361.5625
4753,2020.0,1.0,CHI,MITCHELL TRUBISKY,26.0,QB,QB1,DET,36.0,20.0,...,1.7500,0.4375,0.0000,0.6875,0.0000,0.1875,26.4375,284.4375,115.9375,400.3750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52326,2021.0,17.0,MIA,JASON SANDERS,26.0,PK,PK1,TEN,0.0,0.0,...,1.1875,0.9375,0.0000,0.5000,0.0625,0.1250,27.4375,277.4375,120.8125,398.2500
52327,2021.0,17.0,LAR,MATT GAY,27.0,PK,PK1,BAL,0.0,0.0,...,2.4375,0.6250,0.1250,0.7500,0.0000,0.2500,18.9375,221.0000,108.7500,329.7500
52328,2021.0,17.0,CLE,CHASE MCLAUGHLIN,25.0,PK,PK1,PIT,0.0,0.0,...,3.5000,1.1250,0.0625,0.5625,0.0000,0.1875,19.5000,194.3750,111.4375,305.8125
52329,2021.0,17.0,DAL,GREG ZUERLEIN,34.0,PK,PK1,ARI,0.0,0.0,...,3.0000,0.6875,0.0625,0.6250,0.1250,0.0000,22.9375,226.4375,125.5000,351.9375


In [218]:
# merge all columns needed for model onto mfl_ol_df
df = mfl_ol_df.copy()
# new = pd.DataFrame(columns=histdf.columns[7:])
# df = pd.concat([df, new], axis=1)
# df.loc[:, len(mfl_ol_df.columns):] = 0
df = df.rename(columns={'pos_mfl':'pos'})
df = df.drop(columns='id_mfl')

# Handle duplicates
# Historical duplicates from "allData.csv"
dupList_hist = ['Alex Smith', 'Mike Williams', 'Steve Smith', 'Ryan Griffin', 'Zach Miller', 'David Johnson']
dupList_hist = [x.upper() for x in dupList_hist]
# Handle new year duplicates
dupList = ['JOSH JOHNSON']
dupList.extend(dupList_hist)
# rename duplicate players by appending their position to their name
for p in dupList:
    df.loc[df.player==p, 'player'] = df.loc[df.player==p, 'player'] + df.loc[df.player==p, 'pos']

# Clean age column
df.loc[df.pos=='DF', 'age'] = 0
# assign arbitrary ages to unknown players
df.loc[df.age.isna(), 'age'] = 25
df

,player,pos,team,age,posRank,KR,PR,RES,season
0,BUFFALO BILLS,DF,BUF,0.0,DF1,NO,NO,NO,2022
1,INDIANAPOLIS COLTS,DF,IND,0.0,DF1,NO,NO,NO,2022
2,MIAMI DOLPHINS,DF,MIA,0.0,DF1,NO,NO,NO,2022
3,NEW ENGLAND PATRIOTS,DF,NEP,0.0,DF1,NO,NO,NO,2022
4,NEW YORK JETS,DF,NYJ,0.0,DF1,NO,NO,NO,2022
...,...,...,...,...,...,...,...,...,...
1093,JALEN VIRGIL,WR,DEN,24.0,WR2,NO,NO,NO,2022
1094,DANNY DAVIS,WR,GBP,24.0,WR3,NO,NO,NO,2022
1095,RAMIZ AHMED,PK,GBP,27.0,PK3,NO,NO,NO,2022
1096,PEYTON HENDERSHOT,TE,DAL,23.0,TE3,NO,NO,NO,2022


In [219]:
#Create summary dfs for each season
seasonsPrior1_df = histdf.copy()
seasonsPrior1_df.drop(columns='age', inplace=True)
seasonsPrior1_df = seasonsPrior1_df.loc[seasonsPrior1_df.season==2021]
seasonsPrior1_df['season'] = 2022
seasonsPrior1 = seasonsPrior1_df.groupby(['season', 'player']).mean()
seasonsPrior1['gamesPlayed'] = seasonsPrior1_df.groupby(['season', 'player'])['week'].count()
seasonsPrior1.drop(columns='week', inplace=True)
gamesPlayedCol = seasonsPrior1.pop('gamesPlayed')
seasonsPrior1.insert(0, 'gamesPlayed', gamesPlayedCol)
# Rename all columns in seasonsPrior1
colList = [(x + "_prior1") for x in list(seasonsPrior1.columns)]
seasonsPrior1.columns = colList
seasonsPrior1

# Create summary dfs for combined two seasons before
seasonsPrior2_df = histdf.copy()
seasonsPrior2_df.drop(columns='age', inplace=True)
seasonsPrior2_df['season'] = 2022
seasonsPrior2 = seasonsPrior2_df.groupby(['season', 'player']).mean()
seasonsPrior2['gamesPlayed'] = seasonsPrior2_df.groupby(['season', 'player'])['week'].count()
seasonsPrior2.drop(columns='week', inplace=True)
gamesPlayedCol = seasonsPrior2.pop('gamesPlayed')
seasonsPrior2.insert(0, 'gamesPlayed', gamesPlayedCol)
# Rename all columns in seasonsPrior2
colList = [(x + "_prior2") for x in list(seasonsPrior2.columns)]
seasonsPrior2.columns = colList
seasonsPrior2

gamesPlayed_prior2  passA_prior2  passC_prior2  \
season player                                                          
2022   A.J. BROWN                     26      0.076923      0.000000   
       A.J. GREEN                     26      0.000000      0.000000   
       A.J. MCCARRON                   1      1.000000      1.000000   
       AARON JONES                    29      0.000000      0.000000   
       AARON RODGERS                  31     33.516129     23.354839   
...                                  ...           ...           ...   
       ZACH PASCAL                    30      0.033333      0.000000   
       ZACH WILSON                    12     30.250000     17.166667   
       ZACK MOSS                      25      0.000000      0.000000   
       ZANE GONZALEZ                  24      0.000000      0.000000   
       ZAY JONES                      24      0.041667      0.041667   

                      passY_prior2  passT_prior2  passI_prior2  pass2_prior2  \
season player                                                                  
2022   A.J. BROWN         0.000000      0.000000      0.000000      0.000000   
       A.J. GREEN         0.000000      0.000000      0.000000      0.000000   
       A.J. MCCARRON     20.000000      0.000000      0.000000      0.000000   
       AARON JONES        0.000000      0.000000      0.000000      0.000000   
       AARON RODGERS    266.967742      2.677419      0.290323      0.000000   
...                            ...           ...           ...           ...   
       ZACH PASCAL        0.000000      0.000000      0.000000      0.000000   
       ZACH WILSON      187.250000      0.666667      0.916667      0.083333   
       ZACK MOSS          0.000000      0.000000      0.000000      0.000000   
       ZANE GONZALEZ      0.000000      0.000000      0.000000      0.000000   
       ZAY JONES          1.208333      0.000000      0.000000      0.000000   

                      rushA_prior2  rushY_prior2  rushT_prior2  ...  \
season player                                                   ...   
2022   A.J. BROWN         0.076923      0.384615      0.000000  ...   
       A.J. GREEN         0.000000      0.000000      0.000000  ...   
       A.J. MCCARRON      0.000000      0.000000      0.000000  ...   
       AARON JONES       12.827586     65.620690      0.448276  ...   
       AARON RODGERS      2.258065      7.838710      0.193548  ...   
...                            ...           ...           ...  ...   
       ZACH PASCAL        0.066667      0.700000      0.000000  ...   
       ZACH WILSON        2.250000     13.416667      0.333333  ...   
       ZACK MOSS          8.120000     32.720000      0.320000  ...   
       ZANE GONZALEZ      0.000000      0.000000      0.000000  ...   
       ZAY JONES          0.083333      0.125000      0.000000  ...   

                      defSack_prior1_opp_prior2  defI_prior1_opp_prior2  \
season player                                                             
2022   A.J. BROWN                      2.406250                0.824519   
       A.J. GREEN                      2.206731                0.716346   
       A.J. MCCARRON                   2.687500                0.875000   
       AARON JONES                     2.450431                0.769397   
       AARON RODGERS                   2.433468                0.776210   
...                                         ...                     ...   
       ZACH PASCAL                     2.247917                0.797917   
       ZACH WILSON                     2.093750                0.802083   
       ZACK MOSS                       2.187500                0.907500   
       ZANE GONZALEZ                   2.247396                0.783854   
       ZAY JONES                       2.398438                0.822917   

                      defSaf_prior1_opp_prior2  defFum_prior1_opp_prior2  \
season player                                                       

In [220]:
# Merge df, currentSeason, seasonsPrior1, and seasonsPrior1
final = df.copy()

# Merge df, currentSeason, seasonsPrior1, and seasonsPrior1
final = final.merge(
    seasonsPrior1, how='left', on=['season', 'player']
    ).merge(
        seasonsPrior2, how='left', on=['season', 'player']
        )
# fill NA values for players who do not have two years prior data
final.fillna(0, inplace=True)
final

,player,pos,team,age,posRank,KR,PR,RES,season,gamesPlayed_prior1,...,defSack_prior1_opp_prior2,defI_prior1_opp_prior2,defSaf_prior1_opp_prior2,defFum_prior1_opp_prior2,defBlk_prior1_opp_prior2,defT_prior1_opp_prior2,defPtsAgainst_prior1_opp_prior2,defPassYAgainst_prior1_opp_prior2,defRushYAgainst_prior1_opp_prior2,defYdsAgainst_prior1_opp_prior2
0,BUFFALO BILLS,DF,BUF,0.0,DF1,NO,NO,NO,2022,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,INDIANAPOLIS COLTS,DF,IND,0.0,DF1,NO,NO,NO,2022,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,MIAMI DOLPHINS,DF,MIA,0.0,DF1,NO,NO,NO,2022,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NEW ENGLAND PATRIOTS,DF,NEP,0.0,DF1,NO,NO,NO,2022,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NEW YORK JETS,DF,NYJ,0.0,DF1,NO,NO,NO,2022,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,JALEN VIRGIL,WR,DEN,24.0,WR2,NO,NO,NO,2022,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1094,DANNY DAVIS,WR,GBP,24.0,WR3,NO,NO,NO,2022,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1095,RAMIZ AHMED,PK,GBP,27.0,PK3,NO,NO,NO,2022,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1096,PEYTON HENDERSHOT,TE,DAL,23.0,TE3,NO,NO,NO,2022,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# need to multiply df by each week
# need to get team schedules and build opponent features